<a href="https://colab.research.google.com/github/desmond-ong/pplAffComp/blob/master/Colab/PPLTutorial_4_MVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extensions to a Multimodal VAE

Another way that the VAE model can be extended is to fully let the latent $z$ "cause" both the emotion ratings and the facial expression. This is an example of a Multimodal VAE (Wu & Goodman, 2018).

There is a nice theoretical motivation for this model too. Throughout the past few examples, we've assumed that the space of emotions is exactly what we measured (e.g., some value of happiness, some value of sadness), but maybe the latent space is structured not along these discrete emotion categories, but perhaps along dimensions like "good" vs "bad", or some other semantic property. In emotion theory, this undifferentiated space is called affect, and often, this is low-dimensional (2 to 3 dimensions capture most of the variance in empirical data -- although in practice we often set our latent dimension parameter to be higher in these latent variable models).

We could thus posit a latent *affect* that generates the emotion ratings. That is, the emotion ratings is some projection of the latent affect space onto these emotion concepts that are given meaning by the language and culture that one resides in.

But in fact, we would still want a latent $z$ that captures non-emotional aspects of the face. For simplicity, for this example, we assume that this latent $z$ will capture some aspects of affect as well as the face. Learning to disentangle latent variables (e.g. the latent variables that are important for emotions and the latent variables that are not) is also an active area of research (Narayanaswamy et al, 2017).

And finally, we can add the "outcome to appraisal to affect" part back into this multimodal model.

<div style="width: 300px; margin: auto;"> <center>
<img src="https://desmond-ong.github.io/pplAffComp/code/images/graphicalModel_MVAE.png" width=300></img> </center>
</div>


## Preamble

These first chunks of code clones the GitHub repo, installs Pyro and other requirements on the Colab server (if not already installed), and imports the necessary python packages and functions that we will use.

In [0]:
!git clone https://github.com/desmond-ong/pplAffComp.git
%cd pplAffComp/code
!pip install torch torchvision pyro-ppl

In [0]:
import os
import urllib # for urllib.urlretrieve()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam


from torchvision import transforms, utils, datasets
from torchvision.transforms import ToPILImage
from skimage import io, transform
from PIL import Image
from matplotlib.pyplot import imshow

from pyro.contrib.examples.util import print_and_log
import pyro.poutine as poutine
# custom helperCode for this tutorial, in helperCode.py
import helperCode

import time


print("Currently using Pyro version: " + pyro.__version__)

#IMG_WIDTH = 100
# Note that we downsample to 64 x 64 here, because we wanted a nice power of 2 
#(and DCGAN architecture assumes input image of 64x64) 

IMG_WIDTH = 64
IMG_SIZE = IMG_WIDTH*IMG_WIDTH*3
BATCH_SIZE = 32
DEFAULT_Z_DIM = 50

OUTCOME_VAR_NAMES = ['payoff1', 'payoff2', 'payoff3', 
                     'prob1', 'prob2', 'prob3', 
                     'win', 'winProb', 'angleProp']
EMOTION_VAR_NAMES = ['happy', 'sad', 'anger', 'surprise', 
                     'disgust', 'fear', 'content', 'disapp']

OUTCOME_VAR_DIM = len(OUTCOME_VAR_NAMES)
EMOTION_VAR_DIM = len(EMOTION_VAR_NAMES)

# helper functions
class Swish(nn.Module):
    """https://arxiv.org/abs/1710.05941"""
    def forward(self, x):
        return x * torch.sigmoid(x)

def swish(x):
    return x * torch.sigmoid(x)

#### Dataset

(This part is the same as the SSVAE Tutorial)

We will be using the same dataset as the previous examples. We will consider the trials in which participants only saw a facial expression, and rated how they thought the character feels, or what we call the "facial expression only" trials.

Here is a preview of the 18 faces (which are in ../CognitionData/faces/).

In [0]:
faces_path = os.path.join(os.path.abspath('..'), "CognitionData", "faces")

# initializing two temp arrays
faceArray1 = np.zeros(shape=(100,1,3), dtype='uint8')
faceArray2 = np.zeros(shape=(100,1,3), dtype='uint8')

count = 0
for thisFace in helperCode.FACE_FILENAMES:
    newFaceArray = np.array(Image.open(os.path.join(faces_path, thisFace + ".png")))
    if count < 6 or count > 14:
        faceArray1 = np.concatenate((faceArray1, newFaceArray), axis=1)
    else:
        faceArray2 = np.concatenate((faceArray2, newFaceArray), axis=1)
    count += 1

# concatenating the arrays and removing the first temp column
faceArray = np.concatenate((faceArray1, faceArray2), axis=0)
faceArray = faceArray[:,1:,:]
Image.fromarray(faceArray)

This next chunk defines a function to read in the data, and stores the data in `face_outcome_emotion_dataset`. There are N=1,587 observations, and each observation consists of:

- an accompanying face image
- a 9-dimension outcome vector that parameterizes the gamble that agents played, and
- an 8-dimensional emotion rating vector

In [0]:
# data location
dataset_path = os.path.join(os.path.abspath('..'), "CognitionData", "data_faceWheel.csv")

class FaceOutcomeEmotionDataset(Dataset):
    """Face Outcome Emotion dataset."""
    
    def __init__(self, csv_file, img_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the experiment csv file 
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.expdata = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        
        ## Normalizing the data:
        ####
        ## payoff1, payoff2, payoff3 and win are between 0 and 100
        ## need to normalize to [0,1] to match the rest of the variables,
        ## by dividing payoff1, payoff2, payoff3 and win by 100
        ####
        self.expdata.loc[:,"payoff1"] = self.expdata.loc[:,"payoff1"]/100
        self.expdata.loc[:,"payoff2"] = self.expdata.loc[:,"payoff2"]/100
        self.expdata.loc[:,"payoff3"] = self.expdata.loc[:,"payoff3"]/100
        self.expdata.loc[:,"win"]     = self.expdata.loc[:,"win"]/100
        # Emotions were rated on a 1-9 Likert scale.
        # use emo <- (emo-1)/8 to transform to within [0,1]
        self.expdata.loc[:,"happy":"disapp"] = (self.expdata.loc[:,"happy":"disapp"]-1)/8

    def __len__(self):
        return len(self.expdata)

    def __getitem__(self, idx):
        ratings = np.array(self.expdata.iloc[idx]["happy":"disapp"], np.float32)
        
        outcomes = np.array(self.expdata.iloc[idx]["payoff1":"angleProp"], np.float32)
        
        img_name = os.path.join(self.img_dir, self.expdata.iloc[idx]["facePath"] + ".png")
        try:
            image = Image.open(img_name).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
        except:
            print(img_name)
            raise

        return image, ratings, outcomes

data_transform = transforms.Compose([
    # Note that we downsample to 64 x 64 here, because we wanted a nice power of 2 
    #(and DCGAN architecture assumes input image of 64x64) 
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor()
    ])

# reads in datafile.
print("Reading in dataset...")

face_outcome_emotion_dataset = FaceOutcomeEmotionDataset(csv_file=dataset_path, 
                                                         img_dir=faces_path, 
                                                         transform=data_transform)
face_outcome_emotion_loader = torch.utils.data.DataLoader(face_outcome_emotion_dataset,
                                                          batch_size=BATCH_SIZE, shuffle=True,
                                                          num_workers=4)

N_samples = len(face_outcome_emotion_dataset)
print("Number of observations:", N_samples)

# taking a sample observation
img1, emo1, out1 = face_outcome_emotion_dataset[5]
print("Sample Observation: ")
print(helperCode.EMOTION_VAR_NAMES)
print(emo1)
print(helperCode.OUTCOME_VAR_NAMES)
print(out1)
Image.fromarray(helperCode.TensorToPILImage(img1*255.))

### Defining the encoder and decoder networks:

Here, we define the `Encoder` and `Decoder` modules, basically the two neural networks that go from some modality to $z$, and $z$ to image, respectively.

This is the same as the VAE and SSVAE, just that here, in the *Multimodal* VAE, we have several of these networks. For example, an ImageEncoder/Decoder that we borrow the DCGAN architecture, a Rating Encoder/Decoder and an Outcome Encoder/Decoder. (It's not completely optimized or parameterized, there are a lot of hard-coded decisions here).

In [0]:
class ProductOfExperts(nn.Module):
    """
    Return parameters for product of independent experts.
    See https://arxiv.org/pdf/1410.7827.pdf for equations.

    @param loc: M x D for M experts
    @param scale: M x D for M experts
    """
    def forward(self, loc, scale, eps=1e-8):
        scale = scale + eps # numerical constant for stability
        # precision of i-th Gaussian expert (T = 1/sigma^2)
        T = 1. / scale
        product_loc = torch.sum(loc * T, dim=0) / torch.sum(T, dim=0)
        product_scale = 1. / torch.sum(T, dim=0)
        return product_loc, product_scale
      
class ImageEncoder(nn.Module):
    """
    define the PyTorch module that parametrizes q(z|image).
    This goes from images to the latent z
    
    This is the standard DCGAN architecture.

    @param z_dim: integer
                  size of the tensor representing the latent random variable z
    """
    def __init__(self, z_dim):
        super(ImageEncoder, self).__init__()
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
        #                padding=0, dilation=1, groups=1, bias=True)
        # H_out = floor( (H_in + 2*padding - dilation(kernel_size-1) -1) / stride    +1)
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1, bias=False),
            Swish(),
            nn.Conv2d(32, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            Swish(),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            Swish(),
            nn.Conv2d(128, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            Swish())
        # Here, we define two layers, one to give z_loc and one to give z_scale
        self.z_loc_layer = nn.Sequential(
            nn.Linear(256 * 5 * 5, 512), # it's 256 * 5 * 5 if input is 64x64.
            Swish(),
            nn.Dropout(p=0.1),
            nn.Linear(512, z_dim))
        self.z_scale_layer = nn.Sequential(
            nn.Linear(256 * 5 * 5, 512), # it's 256 * 5 * 5 if input is 64x64.
            Swish(),
            nn.Dropout(p=0.1),
            nn.Linear(512, z_dim))
        self.z_dim = z_dim

    def forward(self, image):
        hidden = self.features(image)
        hidden = hidden.view(-1, 256 * 5 * 5) # it's 256 * 5 * 5 if input is 64x64.
        z_loc = self.z_loc_layer(hidden)
        z_scale = torch.exp(self.z_scale_layer(hidden)) #add exp so it's always positive
        return z_loc, z_scale
    
class ImageDecoder(nn.Module):
    """
    define the PyTorch module that parametrizes p(image|z).
    This goes from the latent z to the images
    
    This is the standard DCGAN architecture.

    @param z_dim: integer
                  size of the tensor representing the latent random variable z
    """
    def __init__(self, z_dim):
        super(ImageDecoder, self).__init__()
        self.upsample = nn.Sequential(
            nn.Linear(z_dim, 256 * 5 * 5),  # it's 256 * 5 * 5 if input is 64x64.
            Swish())
        self.hallucinate = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 1, 0, bias=False),
            nn.BatchNorm2d(128),
            Swish(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            Swish(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            Swish(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias=False))

    def forward(self, z):
        # the input will be a vector of size |z_dim|
        z = self.upsample(z)
        z = z.view(-1, 256, 5, 5) # it's 256 * 5 * 5 if input is 64x64.
        image = self.hallucinate(z) # this is the image
        return image  # NOTE: no sigmoid here. See train.py

      
class RatingEncoder(nn.Module):
    """
    define the PyTorch module that parametrizes q(z|rating).
    This goes from ratings to the latent z

    @param z_dim: integer
                  size of the tensor representing the latent random variable z
    """
    def __init__(self, z_dim):
        super(RatingEncoder, self).__init__()
        self.net = nn.Linear(helperCode.EMOTION_VAR_DIM, 512)
        self.z_loc_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, z_dim))
        self.z_scale_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, z_dim))
        self.z_dim = z_dim

    def forward(self, rating):
        hidden = self.net(rating)
        z_loc = self.z_loc_layer(hidden)
        z_scale = torch.exp(self.z_scale_layer(hidden))
        return z_loc, z_scale


class RatingDecoder(nn.Module):
    """
    define the PyTorch module that parametrizes p(rating|z).
    This goes from the latent z to the ratings

    @param z_dim: integer
                  size of the tensor representing the latent random variable z
    """
    def __init__(self, z_dim):
        super(RatingDecoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 512),
            Swish())
        self.rating_loc_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, helperCode.EMOTION_VAR_DIM))
        self.rating_scale_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, helperCode.EMOTION_VAR_DIM))

    def forward(self, z):
        #batch_size = z.size(0)
        hidden = self.net(z)
        rating_loc = self.rating_loc_layer(hidden)
        rating_scale = torch.exp(self.rating_scale_layer(hidden))
        return rating_loc, rating_scale  # NOTE: no softmax here. See train.py


class OutcomeEncoder(nn.Module):
    """
    define the PyTorch module that parametrizes q(z|outcome).
    This goes from outcomes to the latent z

    @param z_dim: integer
                  size of the tensor representing the latent random variable z
    """
    def __init__(self, z_dim):
        super(OutcomeEncoder, self).__init__()
        self.net = nn.Linear(helperCode.OUTCOME_VAR_DIM, 512)
        self.z_loc_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, z_dim))
        self.z_scale_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, z_dim))
        self.z_dim = z_dim

    def forward(self, outcomes):
        hidden = self.net(outcomes)
        z_loc = self.z_loc_layer(hidden)
        z_scale = torch.exp(self.z_scale_layer(hidden))
        return z_loc, z_scale


class OutcomeDecoder(nn.Module):
    """
    define the PyTorch module that parametrizes p(outcomes|z).
    This goes from the latent z to the outcomes

    @param z_dim: integer
                  size of the tensor representing the latent random variable z
    """
    def __init__(self, z_dim):
        super(OutcomeDecoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 512),
            Swish())
        self.outcome_loc_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, helperCode.OUTCOME_VAR_DIM))
        self.outcome_scale_layer = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Linear(512, helperCode.OUTCOME_VAR_DIM))


    def forward(self, z):
        hidden = self.net(z)
        outcome_loc = self.outcome_loc_layer(hidden)
        outcome_scale = torch.exp(self.outcome_scale_layer(hidden))
        return outcome_loc, outcome_scale  # no nonlinearity here | will be added later


### MVAE Module
Here we define the MVAE Module

In [0]:
class MVAE(nn.Module):
    """
    This class encapsulates the parameters (neural networks), models & guides needed to train a
    multimodal variational auto-encoder.
    Modified from https://github.com/mhw32/multimodal-vae-public
    Multimodal Variational Autoencoder.

    @param z_dim: integer
                  size of the tensor representing the latent random variable z
                  
    Currently all the neural network dimensions are hard-coded; 
    in a future version will make them be inputs into the constructor
    """
    def __init__(self, z_dim, use_cuda=False):
        super(MVAE, self).__init__()
        self.z_dim = z_dim
        self.image_encoder = ImageEncoder(z_dim)
        self.image_decoder = ImageDecoder(z_dim)
        self.rating_encoder = RatingEncoder(z_dim)
        self.rating_decoder = RatingDecoder(z_dim)
        self.outcome_encoder = OutcomeEncoder(z_dim)
        self.outcome_decoder = OutcomeDecoder(z_dim)
        self.experts = ProductOfExperts()
        self.use_cuda = use_cuda
        # relative weights of losses in the different modalities
        self.LAMBDA_IMAGES = 1.0
        self.LAMBDA_RATINGS = 50.0
        self.LAMBDA_OUTCOMES = 50.0
        
        # using GPUs for faster training of the networks
        if self.use_cuda:
            self.cuda()
    
    def model(self, images=None, ratings=None, outcomes=None, annealing_beta=1.0):
        # register this pytorch module and all of its sub-modules with pyro
        pyro.module("mvae", self)
        
        batch_size = 0
        if images is not None:
            batch_size = images.size(0)
        elif ratings is not None:
            batch_size = ratings.size(0)
        elif outcomes is not None:
            batch_size = outcomes.size(0)
        
        with pyro.plate("data"):                        
            # sample from outcome prior N(0.5,0.1), compute p(z|outcome)
            outcome_prior_loc = torch.zeros(torch.Size((batch_size, helperCode.OUTCOME_VAR_DIM))) + 0.5
            outcome_prior_scale = torch.ones(torch.Size((batch_size, helperCode.OUTCOME_VAR_DIM))) * 0.1
            if outcomes is not None:
                # if outcome is provided as an observed input, score against it
                with poutine.scale(scale=self.LAMBDA_OUTCOMES):
                    pyro.sample("obs_outcome", dist.Normal(outcome_prior_loc, outcome_prior_scale), obs=outcomes)
            else:
                # else if outcome is not provided, just sample from priors
                with poutine.scale(scale=self.LAMBDA_OUTCOMES):
                    outcomes = pyro.sample("obs_outcome", dist.Normal(outcome_prior_loc, outcome_prior_scale))
            
            z_loc, z_scale = self.outcome_encoder.forward(outcomes)
            
            # sample from prior (value will be sampled by guide when computing the ELBO)
            with poutine.scale(scale=annealing_beta):
                z = pyro.sample("latent", dist.Normal(z_loc, z_scale))

            # decode the latent code z
            img_loc = self.image_decoder.forward(z)
            # score against actual images
            if images is not None:
                with poutine.scale(scale=self.LAMBDA_IMAGES):
                    pyro.sample("obs_img", dist.Bernoulli(img_loc), obs=images)
            
            rating_loc, rating_scale = self.rating_decoder.forward(z)
            if ratings is not None:
                with poutine.scale(scale=self.LAMBDA_RATINGS):
                    pyro.sample("obs_rating", dist.Normal(rating_loc, rating_scale), obs=ratings)

            # return the loc so we can visualize it later
            return img_loc, rating_loc
    
    def guide(self, images=None, ratings=None, outcomes=None, annealing_beta=1.0):
        # register this pytorch module and all of its sub-modules with pyro
        pyro.module("mvae", self)
        
        batch_size = 0
        if images is not None:
            batch_size = images.size(0)
        elif ratings is not None:
            batch_size = ratings.size(0)
        elif outcomes is not None:
            batch_size = outcomes.size(0)
            
        with pyro.plate("data"):
            # use the encoder to get the parameters used to define q(z|x)
                        
            # initialize the prior expert.
            # we initalize an additional dimension, along which we concatenate all the 
            #   different experts.
            # self.experts() then combines the information from these different modalities
            #   by multiplying the gaussians together
            z_loc = torch.zeros(torch.Size((1, batch_size, self.z_dim))) + 0.5
            z_scale = torch.ones(torch.Size((1, batch_size, self.z_dim))) * 0.1
            if self.use_cuda:
                z_loc, z_scale = z_loc.cuda(), z_scale.cuda()
            
            if outcomes is not None:
                outcome_z_loc, outcome_z_scale = self.outcome_encoder.forward(outcomes)
                z_loc = torch.cat((z_loc, outcome_z_loc.unsqueeze(0)), dim=0)
                z_scale = torch.cat((z_scale, outcome_z_scale.unsqueeze(0)), dim=0)
                
            if images is not None:
                image_z_loc, image_z_scale = self.image_encoder.forward(images)
                z_loc = torch.cat((z_loc, image_z_loc.unsqueeze(0)), dim=0)
                z_scale = torch.cat((z_scale, image_z_scale.unsqueeze(0)), dim=0)
            
            if ratings is not None:
                rating_z_loc, rating_z_scale = self.rating_encoder.forward(ratings)
                z_loc = torch.cat((z_loc, rating_z_loc.unsqueeze(0)), dim=0)
                z_scale = torch.cat((z_scale, rating_z_scale.unsqueeze(0)), dim=0)
            
            z_loc, z_scale = self.experts(z_loc, z_scale)
            # sample the latent z
            with poutine.scale(scale=annealing_beta):
                pyro.sample("latent", dist.Normal(z_loc, z_scale))
    
    def forward(self, image=None, rating=None, outcome=None):
        z_loc, z_scale  = self.infer(image, rating, outcome)
        z = pyro.sample("latent", dist.Normal(z_loc, z_scale).independent(1))
        # reconstruct inputs based on that gaussian
        image_recon = self.image_decoder(z)
        rating_recon = self.rating_decoder(z)
        outcome_recon = self.outcome_decoder(z)
        return image_recon, rating_recon, outcome_recon, z_loc, z_scale

    def infer(self, images=None, ratings=None, outcomes=None):
        batch_size = 0
        if images is not None:
            batch_size = images.size(0)
        elif ratings is not None:
            batch_size = ratings.size(0)
        elif outcomes is not None:
            batch_size = outcomes.size(0)
            
        # initialize the prior expert
        # we initalize an additional dimension, along which we concatenate all the 
        #   different experts.
        # self.experts() then combines the information from these different modalities
        #   by multiplying the gaussians together
        z_loc = torch.zeros(torch.Size((1, batch_size, self.z_dim))) + 0.5
        z_scale = torch.ones(torch.Size((1, batch_size, self.z_dim))) * 0.1
        if self.use_cuda:
            z_loc, z_scale = z_loc.cuda(), z_scale.cuda()

        if outcomes is not None:
            outcome_z_loc, outcome_z_scale = self.outcome_encoder.forward(outcomes)
            z_loc = torch.cat((z_loc, outcome_z_loc.unsqueeze(0)), dim=0)
            z_scale = torch.cat((z_scale, outcome_z_scale.unsqueeze(0)), dim=0)

        if images is not None:
            image_z_loc, image_z_scale = self.image_encoder.forward(images)
            z_loc = torch.cat((z_loc, image_z_loc.unsqueeze(0)), dim=0)
            z_scale = torch.cat((z_scale, image_z_scale.unsqueeze(0)), dim=0)

        if ratings is not None:
            rating_z_loc, rating_z_scale = self.rating_encoder.forward(ratings)
            z_loc = torch.cat((z_loc, rating_z_loc.unsqueeze(0)), dim=0)
            z_scale = torch.cat((z_scale, rating_z_scale.unsqueeze(0)), dim=0)

        z_loc, z_scale = self.experts(z_loc, z_scale)
        return z_loc, z_scale

    # define a helper function for reconstructing images
    def reconstruct_img(self, images):
        # encode image x
        z_loc, z_scale = self.image_encoder(images)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        img_loc = self.image_decoder.forward(z)
        return img_loc

    # define a helper function for reconstructing images without sampling
    def reconstruct_img_nosample(self, images):
        # encode image x
        z_loc, z_scale = self.image_encoder(images)
        ## sample in latent space
        #z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        img_loc = self.image_decoder.forward(z_loc)
        return img_loc
    
    

### Training

Next we define the parameters of our training session, and set up the model and inference algorithm.

Since the training takes a while, `num_epochs` below is set to 2 just to demonstrate training. Ideally you'll want to train for much longer.

In [0]:
pyro.clear_param_store()

class Args:
    learning_rate = 5e-6
    num_epochs = 2 #500
    z_dim = DEFAULT_Z_DIM
    seed = 30
    cuda = False
    
args = Args()

# setup the VAE
mvae = MVAE(z_dim=args.z_dim, use_cuda=args.cuda)

# setup the optimizer
adam_args = {"lr": args.learning_rate}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(mvae.model, mvae.guide, optimizer, loss=Trace_ELBO())

### Training loop

This next chunk of code runs the training over `num_epochs` epochs.

In [0]:
train_elbo = []
trainingTimes = [time.time()]
# training loop
for epoch in range(args.num_epochs):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch returned
    # by the data loader
    for batch_num, (faces, ratings, outcomes) in enumerate(face_outcome_emotion_loader):
        # if on GPU put mini-batch into CUDA memory
        if args.cuda:
            faces, ratings, outcomes = faces.cuda(), ratings.cuda(), outcomes.cuda()
        
        # do ELBO gradient and accumulate loss
        #print("Batch: ", batch_num, "out of", len(train_loader))
        epoch_loss += svi.step(images=faces, ratings=ratings, outcomes=outcomes)
        epoch_loss += svi.step(images=faces, ratings=ratings, outcomes=None)
        epoch_loss += svi.step(images=faces, ratings=None, outcomes=outcomes)
        epoch_loss += svi.step(images=None, ratings=ratings, outcomes=outcomes)
        epoch_loss += svi.step(images=faces, ratings=None, outcomes=None)
        epoch_loss += svi.step(images=None, ratings=ratings, outcomes=None)
        epoch_loss += svi.step(images=None, ratings=None, outcomes=outcomes)

    # report training diagnostics
    normalizer_train = len(face_outcome_emotion_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    train_elbo.append(total_epoch_loss_train)
    # report training diagnostics
    trainingTimes.append(time.time())
    epoch_time = trainingTimes[-1] - trainingTimes[-2]
    print("[epoch %03d]  time: %.2f, average training loss: %.4f" % (epoch, epoch_time, total_epoch_loss_train))
    #if ((epoch+1) % 50 == 0):
        #pyro.get_param_store().save('trained_models/checkpoints/tutorial_mvae_pretrained_' + str(epoch) + '.save')
        

In [0]:
plt.plot(train_elbo)

It takes a while to train (far too slow for a laptop in a tutorial). Thus you can use the following chunks to save or load a model. We assume that you'll skip this save step and load the model from `trained_models/`. It's a large file so you can download it from: https://www.dropbox.com/s/ae157cladfneyxi/mvae_pretrained.save?dl=1 and put it into `trained_models/`

In [0]:
# save model if you decide to modify the above code to train your own model
savemodel = False
if savemodel:
    if not os.path.exists('trained_models'):
      os.mkdir('trained_models')
    pyro.get_param_store().save('trained_models/mvae_pretrained.save')

In [0]:
## making a "trained_models" directory
if not os.path.exists('trained_models'):
    os.mkdir('trained_models')

## use this chunk to download a saved version of the model directly onto colab
downloadmodel = True
if downloadmodel:
    urllib.request.urlretrieve(
        "https://www.dropbox.com/s/ae157cladfneyxi/mvae_pretrained.save?dl=1", 
        "trained_models/mvae_pretrained.save")
    print("Model downloaded")

## use this chunk to help upload the file if you have it on your local machine
#from google.colab import files
#!mkdir trained_models
#%cd trained_models
#files.upload()
#%cd ..

In [0]:
loadmodel = True
if loadmodel:
    pyro.get_param_store().load('trained_models/mvae_pretrained.save')
    pyro.module("mvae", mvae, update_module_params=True)

Now, let's reconstruct some faces! We defined a function, `mvae.reconstruct_img_nosample()`, that allows us to condition on a given image and then reconstructe it. (Right now it's not yet fully optimized to allow sampling)

- In the top row, we show 10 random samples from the training set.
- In the bottom row, we show reconstructions from the faces in the top row.

In [0]:
NUM_SAMPLES = 10
input_array = np.zeros(shape=(IMG_WIDTH, 1, 3), dtype="uint8")
reconstructed_array = np.zeros(shape=(IMG_WIDTH, 1, 3), dtype="uint8")

for batch_num, (faces, ratings, outcomes) in enumerate(face_outcome_emotion_loader):
    # pick NUM_SAMPLES random test images from the first mini-batch and
    # visualize how well we're reconstructing them
    if batch_num == 0:
        reco_indices = np.random.randint(0, faces.size(0), NUM_SAMPLES)
        for index in reco_indices:
            input_img = faces[index, :]
            # storing the input image
            input_img_display = np.array(input_img*255., dtype='uint8')
            input_img_display = input_img_display.transpose((1, 2, 0))
            input_array = np.concatenate((input_array, input_img_display), axis=1)

            # generating the reconstructed image and adding to array
            input_img = input_img.view(1, 3, IMG_WIDTH, IMG_WIDTH)
            reconstructed_img = mvae.reconstruct_img_nosample(input_img)
            reconstructed_img = reconstructed_img.view(3, IMG_WIDTH, IMG_WIDTH).detach().numpy()
            reconstructed_img = np.array(reconstructed_img*255., dtype='uint8')
            reconstructed_img = reconstructed_img.transpose((1, 2, 0))
            reconstructed_array = np.concatenate((reconstructed_array, reconstructed_img), axis=1)

# remove first, blank column, and concatenate
input_array = input_array[:,1:,:]
reconstructed_array = reconstructed_array[:,1:,:]
display_array = np.concatenate((input_array, reconstructed_array), axis=0)
Image.fromarray(display_array)

It's not perfect -- there are still some artifacts in the reconstruction, which will take a bit of tweaking with the parameters (or more data) to fix.

## Coda

In this tutorial, we leveraged a very recent published model (the MVAE from Wu & Goodman, 2018) to train a deep generative model that learns a latent ("affect + face + outcome") space from game outcomes, and their associated emotional faces and ratings.


The model in this particular example is not completely optimized yet, because we are at the boundary of machine learning research, and we are also using a slightly more complicated dataset than say, MNIST or other common pedagogical dataset. We specifically chose an affective computing-relevant dataset so that we could outline psychological hypotheses and then, using probabilistic programming, build some quick models to test these.


We hope that you have enjoyed this tutorial, and that it has been helpful!


-----

Written by: Desmond Ong (desmond.c.ong@gmail.com)

References:

Pyro [VAE tutorial](http://pyro.ai/examples/vae.html)

M. Wu, & N. Goodman. (2018). Multimodal Generative Models for Scalable Weakly-Supervised Learning. In *Advances in Neural Information Processing Systems 31*. 

Hoffman, M. D., Blei, D. M., Wang, C., & Paisley, J. (2013). Stochastic
variational inference. *The Journal of Machine Learning Research*, 14(1),
1303-1347.

Kingma, D. P., & Welling, M. (2014). Auto-encoding variational bayes. Auto-Encoding Variational Bayes. In *The International Conference on Learning Representations*. https://arxiv.org/abs/1312.6114

Data from https://github.com/desmond-ong/affCog, from the following paper:

Ong, D. C., Zaki, J., & Goodman, N. D. (2015). Affective Cognition: Exploring lay theories of emotion. *Cognition*, 143, 141-162.